A feature_engine é uma biblioteca de Python voltada para engenharia de atributos em projetos de machine learning. Ela ajuda a preparar variáveis de forma organizada, sem precisar escrever tudo manualmente.

Usar, por exemplo, para tratamento de valores faltantes: média, mediana, constantes, métodos por categoria

In [0]:
!pip install feature_engine

MÓDULO DE IMPUTAÇÃO E SELEÇÃO!

In [0]:
from feature_engine import imputation, selection

#esc + A acidiona um novo bloco de código

In [0]:
# SEMMA

# SAMPLE

df = spark.table("analitics.olist.abt_ativacao").toPandas()
df.display()

In [0]:
df.head(5)

In [0]:
df['dtRef'].unique()

In [0]:
df_oot = df[df['dtRef'] == df['dtRef'].max()].copy()

df_train = df[df['dtRef'] < df['dtRef'].max()].copy()

In [0]:
from sklearn import model_selection
features = df_train.columns[4:].tolist()
target = 'flagChurn'

X = df_train[features] #se matriz usa maiusculo
y = df_train[target] # vetor usa minusculo

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, 
                                                                    random_state=42, #semente aleatoria
                                                                    test_size=0.2, #pegar 20% pra teste
                                                                    stratify=y) #olha a variavel resposta e vai sorteando de forma proporcional

print("Taxa Treino:", y_train.mean())
print("Taxa Teste:", y_test.mean())

In [0]:
X_train

In [0]:
# Identificação de Missings

train_isna = X_train.isna().sum()
train_isna[train_isna>0]

IMPUTAR UTILIZANDO **PIPELINE**!

In [0]:
features_imput_mean = ['mediaCubagemProduto',
    'mediaAvaliacao168d',
    'mediaAvaliacao336d',
    'mediaAvaliacao',
    'mediaTamanhoNomeProduto',
    'mediaTamanhoDescricaoProduto', 
    'mediaPesoGramas',]

imputer_mean = imputation.MeanMedianImputer(imputation_method='mean', variables=features_imput_mean)

In [0]:
features_imput_0 = [
    'categoriasDistintas',
    'qtdMediaFotos',
    'pctProdutosComFotos',
    'pctProdutosComDescricao',
    'cliUfAC',
    'cliUfAL',
    'cliUfAM',
    'cliUfAP',
    'cliUfBA',
    'cliUfCE',
    'cliUfDF',
    'cliUfES',
    'cliUfGO',
    'cliUfMA',
    'cliUfMG',
    'cliUfMS',
    'cliUfMT',
    'cliUfPA',
    'cliUfPB',
    'cliUfPE',
    'cliUfPI',
    'cliUfPR',
    'cliUfRJ',
    'cliUfRN',
    'cliUfRO',
    'cliUfRR',
    'cliUfRS',
    'cliUfSC',
    'cliUfSE',
    'cliUfSP',
    'cliUfTO',
    'pctNota1',
    'pctNota2',
    'pctNota3',
    'pctNota4',
    'pctNota5',
    'pctNotaBaixa',
    'maiorCategoriaTotalPedidos',
    'shareTopCategorias',
    'pctautomotivo',
    'pctbeleza_saude',
    'pctbrinquedos',
    'pctcama_mesa_banho',
    'pctcool_stuff',
    'pcteletroportateis',
    'pctesporte_lazer',
    'pctferramentas_jardim',
    'pctinformatica_acessorios',
    'pctmoveis_decoracao',
    'pctmoveis_escritorio',
    'pctperfumaria',
    'pctrelogios_presentes',
    'pcttelefonia',
    'pctutilidades_domesticas',
    'pctPedidosEntreguesNoPrazo',
    'qtdCliente',
    'cliNovo',
    'cliPontual',
    'cliRecorrente',
    'qtdProdutosDistintos',
    'maiorCategoriaTotalReceita',
]

imputer_zeros = imputation.ArbitraryNumberImputer(arbitrary_number=0, variables=features_imput_0)

In [0]:
features_imput_1 = [
'pctTendencia1m_2m',
'pctTendencia1m_3m',
'pctTendencia1m_4m',
'pctTendencia1m_5m',
'pctTendencia1m_6m',
'pctTendencia1m_12m',
'mediaQtdeParcelas',
]

imputer_ones = imputation.ArbitraryNumberImputer(arbitrary_number=1, variables=features_imput_1)

In [0]:
flag_missing = [
'maiorCategoriaVenda',
'maiorCategoriaQtdePedidos',
'vendedorRegiao',
]

imputer_flag_missing = imputation.CategoricalImputer(fill_value="MISSING", variables=flag_missing)

In [0]:
features_imput_max = ['qtdeMediaDiasEntrePedidos']

imputer_max = imputation.EndTailImputer(imputation_method='max', tail='right', fold=1, variables=features_imput_max)

In [0]:
features_drop = ['crescimentoD28',
                 'mediaAvaliacao28d',
                 'mediaAvaliacao56d',
                 'mediaAvaliacao84d']

drop_step = selection.DropFeatures(features_to_drop = features_drop)

In [0]:
# Não podemos trabalhar no scikit-learn com categóricos, então temos que ver o que vamos fazer com os objects

X_train.select_dtypes(include='object').describe()

**One Hot Encoder**: cria uma coluna binária para cada categoria. Representa presença ou ausência sem impor ordem entre as classes. É simples e evita suposições sobre relação numérica, mas aumenta muito a dimensionalidade quando há muitas categorias.

**Decision Tree Encoder**: usa modelo de árvore para gerar regras e transformar categorias em números. Captura padrões complexos.

**Mean Encoder**: usa a média do alvo por categoria. É mais simples e rápido, mas precisa de regularização.

In [0]:
from feature_engine import encoding

features_tree_encoder = ['maiorCategoriaVenda']
tree_encoder = encoding.DecisionTreeEncoder(variables = features_tree_encoder, regression=False)

features_mean_encoder = ['maiorCategoriaQtdePedidos', 'vendedorDescUF', 'vendedorRegiao']
mean_encoder = encoding.MeanEncoder(variables = features_mean_encoder)

features_onehot_encoder = [
    'Tendencia1m_2m',
    'Tendencia1m_3m',
    'Tendencia1m_4m',
    'Tendencia1m_5m',
    'Tendencia1m_6m',
    'Tendencia1m_12m',]
onehot_encoder = encoding.OneHotEncoder(variables = features_onehot_encoder)


CONECTAR AS ETAPAS DE IMPUTS - PIPELINE

In [0]:
from sklearn import pipeline

# aqui a lista espera uma tupla
meu_pipeline = pipeline.Pipeline(steps=[
  ("Imput Media", imputer_mean),
  ("Imput Zeros", imputer_zeros),
  ("Imput Uns", imputer_ones),
  ("Imput Max", imputer_max),
  ("Imput Categorias Missing", imputer_flag_missing),
  ("Remocao", drop_step),
  ("Tree Encoder", tree_encoder),
  ("Mean Encoder", mean_encoder),
  ("OneHot Encoder", onehot_encoder),
  ])

# treinado
meu_pipeline.fit(X_train, y_train)

In [0]:
# transformado

X_train_transform = meu_pipeline.transform(X_train)
X_train_transform.isna().sum().sort_values(ascending=False)

In [0]:
# Sem variáveis categóricas agora
X_train_transform.select_dtypes(include='object').describe()

In [0]:
X_train_transform.shape[1]

In [0]:
X_train_transform.columns